In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import DataLoader
from torchaudio.datasets import LIBRISPEECH
from torchaudio.transforms import MelSpectrogram, Resample
import os

In [3]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

##todo:
splitat u validation i u train set

In [4]:
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
12.1
1
NVIDIA GeForce GTX 1660


In [68]:
# Define a custom transform (Mel spectrogram transformation)
transform = MelSpectrogram(
    sample_rate=16000,
    n_fft=400,        # 25 ms window
    hop_length=160,   # 10 ms stride
    n_mels=40
)

ako treba If you still need nn.Sequential later, wrap it like:

transform = nn.Sequential(
    MelSpectrogram(sample_rate=16000, n_fft=400, hop_length=160, n_mels=40)
)

In [69]:
# Example: simple character map (extend as needed)
char_map = {c: i+1 for i, c in enumerate("abcdefghijklmnopqrstuvwxyz '")}
char_map['<blank>'] = 0
output_size = len(char_map)
def text_to_int_sequence(text):
    return [char_map.get(c, 0) for c in text.lower()]

### što je dropout, zašto

promjena max poola na (2,1)
 Why we also changed stride=(2, 1)?
We usually want stride = kernel size in pooling.

This avoids overlapping windows and makes the output size predictable.

If you used:

python
Kopiraj
Uredi
kernel_size=(2, 2), stride=(2, 1)
You’d pool in time — but only move forward 1 step

That would result in overlapping pooling in time — more expensive

In [70]:
FIXED_INPUT_LENGTH = 8000  # or another value that fits your dataset
FIXED_TIME_STEPS = 1024  # Adjust this value based on your MelSpectrogram output and model
class CNN_LSTM_Model(nn.Module):
    def __init__(self, hidden_size=128, num_layers=2, output_size=len(char_map)):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.dropout = nn.Dropout(0.0)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size

        # ⚠️ Dummy input to infer shape ovako mozemo kasnije promijeniti
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 40, FIXED_TIME_STEPS)  # [B, C, Freq, Time]
            x = self.pool(torch.relu(self.bn1(self.conv1(dummy))))
            x = torch.relu(self.bn2(self.conv2(x)))
            _, channels, freq, _ = x.shape
            lstm_input_size = channels * freq

        self.lstm = nn.LSTM(
            input_size=lstm_input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(self.hidden_size, self.output_size)



    def forward(self, x):
        if len(x.shape) == 3:
            x = x.unsqueeze(1)
        #print("Input to conv1:", x.shape)
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        #print("After pool1:", x.shape)
        x = torch.relu(self.bn2(self.conv2(x)))
        #print("After conv2:", x.shape)
        batch, channels, freq, time = x.shape
        x = x.permute(0, 3, 1, 2).contiguous().view(batch, time, channels * freq)
        #print("After reshape for LSTM:", x.shape)

        lstm_out, _ = self.lstm(x)
        x = self.dropout(self.fc(lstm_out))
        x = x.permute(1, 0, 2)  # [time, batch, classes]
        return x

In [71]:
# Define CTC Loss function
ctc_loss = nn.CTCLoss(blank=0, zero_infinity=True)

In [72]:
class LibriSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, subset="train-clean-100", transform=None, max_duration=10.0):
        self.root_dir = root_dir
        self.subset = subset
        self.transform = transform
        self.data = []

        max_chars = 20  # You can lower to 30 for overfitting tests

        subset_dir = os.path.join(self.root_dir, subset)
        for speaker in os.listdir(subset_dir):
            speaker_path = os.path.join(subset_dir, speaker)
            if os.path.isdir(speaker_path):
                for chapter in os.listdir(speaker_path):
                    chapter_path = os.path.join(speaker_path, chapter)
                    if os.path.isdir(chapter_path):
                        trans_file = os.path.join(chapter_path, f"{speaker}-{chapter}.trans.txt")
                        audio_files = [f for f in os.listdir(chapter_path) if f.endswith('.flac')]
                        
                        # Load all transcriptions
                        trans_dict = {}
                        with open(trans_file, 'r') as f:
                            for line in f:
                                parts = line.strip().split(' ', 1)
                                if len(parts) == 2:
                                    trans_dict[parts[0]] = parts[1].lower()

                        for audio_file in audio_files:
                            audio_path = os.path.join(chapter_path, audio_file)
                            waveform, sample_rate = torchaudio.load(audio_path)
                            duration = waveform.shape[1] / sample_rate
                            utt_id = audio_file.replace('.flac', '')
                            transcription = trans_dict.get(utt_id, "")
                            if duration <= max_duration and len(transcription) <= max_chars:
                                self.data.append({
                                    'audio': audio_path,
                                    'transcription': transcription
                                })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        waveform, sample_rate = torchaudio.load(sample['audio'])
        waveform = waveform.squeeze(0)
        if self.transform:
            spectrogram = self.transform(waveform)
            spectrogram = spectrogram.unsqueeze(0)
            return spectrogram, sample['transcription']
        return waveform.unsqueeze(0), sample['transcription']


FIXED_TIME_STEPS = 512, povećao s 128, jer onda je bilo pre malo output character predictiona, a pošto sam povećao moguć input da proširim dataset
onda duže rečenice fizički nisu mogle biti reprezentirane


In [73]:
import torch
import torch.nn.functional as F
FIXED_TIME_STEPS = FIXED_TIME_STEPS

def pad_waveform(waveform, target_length=FIXED_INPUT_LENGTH):
    # waveform shape: [channels, time] or [1, time]
    waveform_length = waveform.size(1)
    if waveform_length < target_length:
        padding = target_length - waveform_length
        waveform = F.pad(waveform, (0, padding))
    elif waveform_length > target_length:
        waveform = waveform[:, :target_length]
    return waveform


def resize_spec(spec, target_width=FIXED_TIME_STEPS):
    current_width = spec.shape[-1]
    if current_width < target_width:
        return F.pad(spec, (0, target_width - current_width))
    else:
        return spec[..., :target_width]
def collate_fn(batch):
    spectrograms = [resize_spec(item[0]) for item in batch]
    transcripts = [item[1] for item in batch]

    spectrograms = torch.stack(spectrograms)  # shape: [B, 1, 40, FIXED_TIME_STEPS]

    # Targets
    targets = []
    target_lengths = []
    for t in transcripts:
        int_seq = text_to_int_sequence(t)
        targets.extend(int_seq)
        target_lengths.append(len(int_seq))

    targets = torch.tensor(targets, dtype=torch.long)
    target_lengths = torch.tensor(target_lengths, dtype=torch.long)

    return spectrograms, targets, target_lengths


## DONT FORGET TO RELOAD DATALOADER IF YOU FILTER FOR LENGTH ILI NEŠTO DRUGO

In [34]:
dataset = LibriSpeechDataset(
    root_dir=r"X:\AIx\PROJECTS\voiceToText\mojModel\data\LibriSpeech",
    subset="train-clean-100",
    transform=transform
)


In [74]:
print(f"Total filtered samples: {len(dataset)}")

Total filtered samples: 41


In [75]:
train_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [38]:
#model = CNN_LSTM_Model(hidden_size=128, output_size=40)
    # Run one batch through model to initialize LSTM/FC
#inputs, _, _ = next(iter(train_loader))
#_ = model(inputs.to(torch.float32))
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [78]:
model = CNN_LSTM_Model(hidden_size=64, num_layers=1, output_size=output_size).to(device)
# Run one batch through model to initialize LSTM/FC
inputs, _, _ = next(iter(train_loader))
print("Shape before model:", inputs.shape)  # should be [B, 1, 40, T]
_ = model(inputs.to(torch.float32).to(device))
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2) # ovo je za overfitting test
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)



Shape before model: torch.Size([2, 1, 40, 1024])


In [41]:
#!pip install tqdm

In [79]:
inv_char_map = {v: k for k, v in char_map.items()}

def decode_prediction(pred):
    # Collapse repeated characters and remove blanks (0)
    decoded = []
    prev = -1
    for p in pred:
        if p != prev and p != 0:
            decoded.append(inv_char_map.get(p, '?'))
        prev = p
    return ''.join(decoded)

def decode_target(target_seq):
    return ''.join(inv_char_map.get(t, '?') for t in target_seq if t != 0)

In [43]:
!pip install jiwer


treba fine tuneat, ako je target_length veći od output_time_steps ni dobro

napravit cu i ovo

self.pool = nn.MaxPool2d((2, 2))  # ⛔ reduces both time & freq
This halves time resolution.

➡️ Change it to:

self.pool = nn.MaxPool2d((2, 1))  # ✅ only reduces frequency, keeps time
This preserves more time steps → longer output.

🔧 Option C: Combine both (best)


In [80]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
from jiwer import wer, cer

"""
epoch_losses = []  # stores average loss per epoch
epoch_cers = [] # stores average CER per epoch
for epoch in range(80):
    running_loss = 0.0
    cer_scores = []
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        inputs, targets, target_lengths = batch
        inputs = inputs.to(torch.float32).to(device)
        targets = targets.to(device)
        target_lengths = target_lengths.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        # da vidimo output koliki je a koliki treba biti
        print("Output shape:", outputs.shape)  # shape: [time, batch, vocab_size]
        print("Target lengths:", target_lengths.tolist())  # List of target lengths


        # Decode predictions and targets
        preds = outputs.argmax(-1).T  # shape: [batch, time]
        target_offset = 0
        for i in range(preds.shape[0]):
            pred_str = decode_prediction(preds[i].tolist())
            true_len = target_lengths[i].item()
            target_str = decode_target(targets[target_offset:target_offset + true_len].tolist())
            target_offset += true_len
            cer_score = cer(target_str, pred_str)
            cer_scores.append(cer_score)
            if i < 1:  # Only print one sample per batch for readability
                print(f"\nWanted:    {target_str}")
                print(f"Predicted: {pred_str if pred_str else '[BLANK]'}")
                print(f"CER:       {cer_score:.2f}")


        input_lengths = torch.full(size=(inputs.size(0),), fill_value=outputs.size(0), dtype=torch.long, device=device)
        loss = ctc_loss(outputs, targets, input_lengths, target_lengths)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        tqdm.write(f"Epoch {epoch+1} Batch {batch_idx+1} Loss: {loss.item():.4f}")
    average_loss = running_loss / (batch_idx + 1)
    epoch_losses.append(average_loss)
    average_cer = sum(cer_scores) / len(cer_scores)
    epoch_cers.append(average_cer)

    clear_output(wait=True)
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(epoch_losses, marker='o')
    plt.title("CTC Loss Over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Average Loss")
    plt.grid(True)
    plt.ticklabel_format(useOffset=False, style='plain', axis='y')

    plt.subplot(1, 2, 2)
    plt.plot(epoch_cers, marker='x', color='orange')
    plt.title("CER Over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Character Error Rate")
    plt.grid(True)
    plt.ticklabel_format(useOffset=False, style='plain', axis='y')


    plt.tight_layout()
    plt.show()


    print(f"Epoch {epoch + 1} Average Loss: {average_loss:.4f}")
    print(f"Epoch {epoch + 1} Average CER:  {average_cer:.4f}")

    scheduler.step()
"""

'\nepoch_losses = []  # stores average loss per epoch\nepoch_cers = [] # stores average CER per epoch\nfor epoch in range(80):\n    running_loss = 0.0\n    cer_scores = []\n    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):\n        inputs, targets, target_lengths = batch\n        inputs = inputs.to(torch.float32).to(device)\n        targets = targets.to(device)\n        target_lengths = target_lengths.to(device)\n        optimizer.zero_grad()\n\n        outputs = model(inputs)\n        # da vidimo output koliki je a koliki treba biti\n        print("Output shape:", outputs.shape)  # shape: [time, batch, vocab_size]\n        print("Target lengths:", target_lengths.tolist())  # List of target lengths\n\n\n        # Decode predictions and targets\n        preds = outputs.argmax(-1).T  # shape: [batch, time]\n        target_offset = 0\n        for i in range(preds.shape[0]):\n            pred_str = decode_prediction(preds[i].tolist())\n            true_len

In [91]:
debug_batch = next(iter(train_loader))

In [92]:
inputs, targets, target_lengths = debug_batch

# Use the first sample
inputs = inputs[0].unsqueeze(0)  # [1, 1, 40, T]
first_len = target_lengths[0].item()
targets = targets[:first_len]   # take the first `N` characters
target_lengths = target_lengths[0].unsqueeze(0)  # shape: [1]

# Reassign
debug_batch = [inputs, targets, target_lengths]

In [93]:
print("[DEBUG] Target length:", target_lengths.item())
print("[DEBUG] Target indices:", targets.tolist())
print("[DEBUG] Decoded target:", decode_target(targets.tolist()))

[DEBUG] Target length: 19
[DEBUG] Target indices: [19, 1, 9, 14, 20, 27, 2, 18, 9, 4, 5, 28, 19, 27, 5, 9, 7, 8, 20]
[DEBUG] Decoded target: saint bride's eight


In [96]:
model.train()
for step in range(3000):
    inputs, targets, target_lengths = debug_batch
    inputs = inputs.to(torch.float32).to(device)
    targets = targets.to(device)
    target_lengths = target_lengths.to(device)

    optimizer.zero_grad()
    #print("[DEBUG] Spectrogram mean:", inputs.mean().item())
    #print("[DEBUG] Spectrogram std: ", inputs.std().item())
    outputs = model(inputs)
    #print("[DEBUG] Output logits mean:", outputs.mean().item())
    #print("[DEBUG] Output logits std: ", outputs.std().item())
    #print("[DEBUG] Output argmax (first 10 tokens of sample 0):", outputs.argmax(-1)[:10, 0].tolist())
    log_probs = torch.nn.functional.log_softmax(outputs, dim=-1)
    input_lengths = torch.full((inputs.size(0),), outputs.size(0), dtype=torch.long, device=device)
    loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
    input_lengths = torch.full((inputs.size(0),), outputs.size(0), dtype=torch.long, device=device)
    loss.backward()
    optimizer.step()

    #print(f"[DEBUG] Step {step+1}, Loss: {loss.item():.4f}")

    # Decode prediction every N steps
    if step % 50 == 0:
        preds = outputs.argmax(-1).T  # [batch, time]
        target_offset = 0
        for i in range(preds.shape[0]):
            pred_str = decode_prediction(preds[i].tolist())
            true_len = target_lengths[i].item()
            target_str = decode_target(targets[target_offset:target_offset + true_len].tolist())
            target_offset += true_len
            print(f"\n[DEBUG] Step {step+1}")
            print(f"Target:    {target_str}")
            print(f"Predicted: {pred_str if pred_str else '[BLANK]'}")



[DEBUG] Step 1
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 51
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 101
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 151
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 201
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 251
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 301
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 351
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 401
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 451
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 501
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 551
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 601
Target:    saint bride's eight
Predicted: saint bridt

[DEBUG] Step 651
Target:    saint bride's eight
Predicted: saint b